# Creating Custom Feature Maps for Quantum Support Vector Machines


Support vector machines (SVM) address the problem of supervised learning through the construction of a classifier. Havlicek *et al*. proposed two strategies to design a quantum SVM, namely the Quantum Kernel Estimator and the Quantum Variational Classifier. Both of these strategies use data that is provided classically and encodes it in the quantum state space through a quantum feature map [1]. The choice of which feature map to use is important and may depend on the given dataset we want to classify. In this tutorial, we show how to configure new feature maps in Aqua and explore their impact on the accuracy of the quantum classifier.

Aqua provides several options for customizing the quantum feature map. In particular, there are four main parameters that can be used for model selection: the feature map circuit depth, the data map function for encoding the classical data, the quantum gate set and the order of expansion. We will go through each of these parameters in this tutorial, but before getting started, let us review the main concepts of the quantum feature map discussed in [1].

[1] Havlicek _et al_.  Nature **567**, 209-212 (2019). https://www.nature.com/articles/s41586-019-0980-2, https://arxiv.org/abs/1804.11326


### Review of the Quantum Feature Map


A quantum feature map nonlinearly maps a classical datum **x** to a quantum state $|\Phi(\mathbf{x})\rangle\langle\Phi(\mathbf{x})|$, a vector in the Hilbert space of density matrices. Support vector machine classifiers find a hyperplane separating each vector $|\Phi(\mathbf{x}_i)\rangle\langle\Phi(\mathbf{x}_i)|$ depending on its label, supported by a reduced amount of vectors (the so-called support vectors). A key element of the feature map is not only the use of quantum state space as a feature space but also the way data are mapped into this high dimensional space.

Constructing feature maps based on quantum circuits that are hard to simulate classically is an important step towards obtaining a quantum advantage over classical approaches. The authors of [1] proposed a family of feature maps that is conjectured to be hard to simulate classically and that can be implemented as short-depth circuits on near-term quantum devices. The quantum feature map of depth $d$ is implemented by the unitary operator 

$$ \mathcal{U}_{\Phi(\mathbf{x})}=\prod_d U_{\Phi(\mathbf{x})}H^{\otimes n},\ U_{\Phi(\mathbf{x})}=\exp\left(i\sum_{S\subseteq[n]}\phi_S(\mathbf{x})\prod_{k\in S} P_k\right), $$

which contains layers of Hadamard gates interleaved with entangling blocks encoding the classical data as shown in circuit diagram below for $d=2$.

<img src="images/uphi.PNG" width="400" />

The number of qubits $n$ in the quantum circuit is equal to the dimensionality of the classical data $\mathbf{x}$, which are encoded through the coefficients $\phi_S(\mathbf{x})$, where $S \subseteq[n] = \{1, \ldots, n \}$. We call the $r$-th order expansion the feature map of this circuit family when $|S|\leq r$. In Aqua, the default is the second order expansion $|S|\leq 2$ used in [1], which gives $n$ singeltons $S=\{i\}$ and, depending on the connectivity graph of the quantum device, up to $\frac{n(n-1)}{2}$ couples to encode non-linear interactions. The greater the upper bound $r$, the more interactions will be taken into account.

Only contributions from $Z$ and $ZZ$ gates in the entangling blocks are considered in [1]. In general, the blocks can be expressed in terms of the Pauli gates $P_k \in \{\mathbb{1}_k, X_k, Y_k, Z_k \}$.

In Aqua, the circuit depth $d$, coefficients $\phi_S$, expansion order $r$, and gates $P_k$ are mutable for both classification algorithms (Quantum Variational Classifier and Quantum Kernel Estimator). As discussed in [1], the depth $d=1$ circuit can be efficiently simulated classically by uniform sampling, while the $d=2$ variant is conjectured to be hard to simulate classically.

### Programming the Quantum Feature Map

We will now see how to configure quantum feature maps in Aqua by modifing the circuit depth $d$, data map function $\phi_S$, expansion order $r$, and gates $P_k$. Documentation on the quantum feature maps in Aqua can be found at https://qiskit.org/documentation/aqua/feature_maps.html. To configure and compare different feature maps, we will use synthetic data from `datasets.py`, which is generated by the `SecondOrderExpansion` feature map with default settings. As a result, we expect high classification accuracy when training the model with this same feature map. 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import functools

from qiskit import BasicAer
from qiskit.aqua import run_algorithm, QuantumInstance
from qiskit.aqua.components.feature_maps import SecondOrderExpansion, FirstOrderExpansion, PauliExpansion, self_product
from qiskit.aqua.algorithms import QSVM
from datasets import *

In [2]:
# Generate synthetic training and test sets from the SecondOrderExpansion quantum feature map
feature_dim = 2
sample_Total, training_dataset, test_dataset, class_labels = ad_hoc_data(training_size=20, test_size=10,
                                                                         n=feature_dim, gap=0.3,
                                                                         PLOT_DATA=False)

# Using the statevector simulator
backend = BasicAer.get_backend('statevector_simulator')
random_seed = 10598

quantum_instance = QuantumInstance(backend, seed=random_seed, seed_transpiler=random_seed)

With this synthetic data, we will use the Quantum Kernel Estimator to test different feature maps, starting with a first order expansion of the feature map discussed in [1]. From there, we will explore more complex feature maps with higher order expansions and custom functions to map the classical data.

#### 1. First Order Diagonal Expansion



A first order diagonal expansion is implemented using the `FirstOrderExpansion` feature map where $|S|=1$. The resulting circuit contains no interactions between features of the encoded data, and therefore no entanglement. The feature map can take the following inputs:

- `feature_dimension`: dimensionality of the classical data (equal to the number of required qubits)
- `depth`: number of times $d$ to repeat the feature map circuit 
- `data_map_func`: function $\phi_S(\mathbf{x})$ encoding the classical data.

The default setting `data_map_func = self_product` for the `FirstOrderExpansion` has $S = \{i\}$ and is given by 

$$\phi_S:x\mapsto x_i.$$


In [3]:
# Generate the feature map
feature_map = FirstOrderExpansion(feature_dimension=feature_dim, depth=2)

# Run the Quantum Kernel Estimator and classify the test data
qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.3


We see that the first order expansion feature map yields poor classification accuracy on data generated to be separable by the second order expansion.

#### 2. Second Order Diagonal Expansion

The `SecondOrderExpansion` feature map allows $|S|\leq 2$, so interactions in the data will be encoded in the feature map according to the connectivity graph and the classical data map. `SecondOrderExpansion` with default parameters is equivalent to the feature map described in [1] and can take the additional inputs:

- `entangler_map`: encodes qubit connectivity (default `None` uses a precomputed connectivity graph according to `entanglement`) 
- `entanglement`: generates connectivity `'full'` or `'linear'` if `entangler_map` not provided (default value `'full'` indicates a complete connectivity graph of $\frac{n(n-1)}{2}$ interactions)

The default setting for `data_map_func` in `SecondOrderExpansion` is given by

$$\phi_S:x\mapsto \Bigg\{\begin{array}{ll}
    x_i & \mbox{if}\ S=\{i\} \\
        (\pi-x_i)(\pi-x_j) & \mbox{if}\ S=\{i,j\}
    \end{array}$$.


In [4]:
feature_map = SecondOrderExpansion(feature_dimension=feature_dim, depth=2)

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  1.0


As expected, the second order feature map yields high test accuracy on this dataset.

#### 3. Second Order Diagonal Expansion with Custom Data Map

Instead of using the default data map $\phi_S(\mathbf{x})$ in Aqua, we can encode the classical data using custom functions. For example, we will create the following map for our data (shown for $|S| \le 2$, but defined similarly for higher order terms):

$$\phi_S:x\mapsto \Bigg\{\begin{array}{ll}
    x_i & \mbox{if}\ S=\{i\} \\
        \sin(\pi-x_i)\sin(\pi-x_j) & \mbox{if}\ S=\{i,j\}
    \end{array}$$

In [5]:
def custom_data_map_func(x):
    """Define a function map from R^n to R.
    
    Args:
        x (np.ndarray): data
    Returns:
        double: the mapped value
    """
    coeff = x[0] if len(x) == 1 else \
        functools.reduce(lambda m, n: m * n, np.sin(np.pi - x))
    return coeff

Let us now test this custom data map on the synthetic dataset.

In [6]:
feature_map = SecondOrderExpansion(feature_dimension=feature_dim, depth=2, data_map_func=custom_data_map_func)

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.5


We see that this choice for the data map function reduced the accuracy of the model.

#### 4. Second Order Pauli Expansion


For some applications, we may want to consider a more general form of the feature map. One way to generalize is to use `PauliExpansion` and specify a set of Pauli gates instead of using the default $Z$ gates. This feature map has the same parameters as `FirstOrderExpansion` and `SecondOrderExpansion` such as `depth` and `data_map_function` along with an additional `paulis` parameter to change the gate set. This parameter is a list of strings, each representing the desired Pauli gate(s). The default value is `['Z', 'ZZ']`, which is equivalent to `SecondOrderExpansion`.


Each string in `paulis` is implemented one at a time for each layer in the depth $d$ feature map circuit. A single character, for example `'Z'`, adds one layer of the corresponding single-qubit gates, while terms such as `'ZZ'` or `'XY'` add a layer of corresponding two-qubit entangling gates for each qubit pair available.

For example, the choice `paulis = ['Z', 'Y', 'ZZ']` generates a quantum feature map of the form 

$$\mathcal{U}_{\Phi(\mathbf{x})} = \left( \exp\left(i\sum_{jk} \phi_{\{j,k\}}(\mathbf{x}) \, Z_j \otimes Z_k\right) \, \exp\left(i\sum_{j} \phi_{\{j\}}(\mathbf{x}) \, Y_j\right) \, \exp\left(i\sum_j \phi_{\{j\}}(\mathbf{x}) \, Z_j\right) \, H^{\otimes n} \right)^d.$$ 

The depth $d=1$ version of this quantum circuit is shown in the figure below for $n=2$ qubits.

<br>
<img src="images/depth1.PNG" width="400"/>
<br>

The circuit begins with a layer of Hadamard gates $H^{\otimes n}$ followed by a layer of single-qubit $A = e^{i\phi_{\{j\}}(\mathbf{x})Z_j}$ gates and a layer of $B = e^{i\phi_{\{j\}}(\mathbf{x}) \, Y_j}$ gates. The $A$ and $B$ gates are parametrized by the same set of angles $\phi_{\{j\}}(\mathbf{x})$ but around different axes. The diagonal entangling gate $e^{i \phi_{\{0,1\}}(\mathbf{x}) \, Z_0 \otimes Z_1}$ is parametrized by an angle $\phi_{\{0,1\}}(\mathbf{x})$ and can be implemented using two controlled-NOT gates and one $A'=e^{i\phi_{\{0,1\}}(x)\, Z_1}$ gate as shown in the figure.

As a comparison, `paulis = ['Z', 'ZZ']` creates the same circuit as above but without the $B$ gates, while `paulis = ['Z', 'YY']` creates a circuit with a layer of $A$ gates followed by a layer of entangling gates $e^{i \phi_{\{0,1\}}(\mathbf{x}) \, Y_0 \otimes Y_1}$.

Below, we test the `PauliExpansion` with `paulis=['Z', 'Y', 'ZZ']`.

In [7]:
feature_map = PauliExpansion(feature_dimension=feature_dim, depth=2, paulis = ['Z','Y','ZZ'])

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.7


#### 5. Third Order Pauli Expansion with Custom Data Map

Third order or higher expansions can be configured using `PauliExpansion`. For example, assuming the classical data has dimensionality of at least three and we have access to three qubits, `paulis = ['Y', 'Z', 'ZZ', 'ZZZ']` generates a feature map according to the previously mentioned rule, with $|S|\leq 3$. 

Suppose we want to classify data with three features using a third order expansion, a custom data map, and a circuit of depth $d=2$. We can do this with the following code in Aqua.

In [8]:
feature_dim = 3
sample_Total_b, training_dataset_b, test_dataset_b, class_labels = ad_hoc_data(training_size=20, test_size=10, 
                                                                               n=feature_dim, gap=0.3, 
                                                                               PLOT_DATA=False)

In [9]:
feature_map = PauliExpansion(feature_dimension=feature_dim, depth=2, 
                             paulis = ['Y','Z','ZZ','ZZZ'], data_map_func=custom_data_map_func)

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset_b, test_dataset=test_dataset_b)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.6


The qubit connectivity is `'full'` by default, so each layer of this depth $d=2$ circuit will contain the sequence: 

- One layer of $B = e^{i\phi_{\{j\}}(\mathbf{x})\,Y_j}$ gates followed by one layer of $A = e^{i\phi_{\{j\}}(\mathbf{x})\,Z_j}$ gates
- One layer containing a $ZZ$ entangler $e^{i \phi_{\{j,k\}}(\mathbf{x}) \,Z_j \otimes Z_k}$ for each pair of qubits $(0,1),\ (1,2),\ (0,2)$
- One layer containing a $ZZZ$ entangler $e^{i\phi_{\{0,1,2 \}}(x)\,Z_0 \otimes Z_1 \otimes Z_2}$ where $\phi_{\{jkl\}} = \sin(\pi-x_j)\sin(\pi-x_k)\sin(\pi-x_l)$

### Building New Feature Maps


In this tutorial, we have seen how to generate feature maps from the circuit family described in [1]. To explore new circuit families, we can create a new class implementing the class `FeatureMap`, and its method `construct_circuit`, and the new feature map will be pluggable in any Aqua component requiring a feature map. More information on adding new feature maps can be found in the documentation https://qiskit.org/documentation/aqua/feature_maps.html.

As an example to illustrate the process, below we show a general custom feature map class with the circuit construction method that creates a quantum circuit consisting of successive layers of $R_X$ gates and $ZZ$ gates.

In [10]:
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit import QuantumCircuit, QuantumRegister

class CustomFeatureMap(FeatureMap):
    """Mapping data with a custom feature map."""
    
    def __init__(self, feature_dimension, depth=2, entangler_map=None):
        """
        Args:
            feature_dimension (int): number of features
            depth (int): the number of repeated circuits
            entangler_map (list[list]): describe the connectivity of qubits, each list describes
                                        [source, target], or None for full entanglement.
                                        Note that the order is the list is the order of
                                        applying the two-qubit gate.        
        """
        self._feature_dimension = feature_dimension
        self._num_qubits = self._feature_dimension = feature_dimension
        self._depth = depth
        self._entangler_map = None
        if self._entangler_map is None:
            self._entangler_map = [[i, j] for i in range(self._feature_dimension) for j in range(i + 1, self._feature_dimension)]
            
    def construct_circuit(self, x, qr, inverse=False):
        """Construct the feature map circuit.
        
        Args:
            x (numpy.ndarray): 1-D to-be-transformed data.
            qr (QauntumRegister): the QuantumRegister object for the circuit.
            inverse (bool): whether or not to invert the circuit.
            
        Returns:
            QuantumCircuit: a quantum circuit transforming data x.
        """
        qc = QuantumCircuit(qr)

        
        for _ in range(self._depth):
            for i in range(self._feature_dimension):
                qc.rx(x[i], qr[i])
            for [source, target] in self._entangler_map:
                qc.cx(qr[source], qr[target])
                qc.u1(x[source] * x[target], qr[target])
                qc.cx(qr[source], qr[target])
                    
        if inverse:
            qc.inverse()
        return qc


In [11]:
feature_map = CustomFeatureMap(feature_dimension=2, depth=2)

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.55
